In [1]:
import openai

key=open("key.txt").read().strip()
client = openai.OpenAI(api_key=key)

In [2]:
import pyrogram
import pickle

pkl = pickle.load(open('data/messages.pkl', 'rb'))
history = pkl["history"]
msgs = pkl["msgs"]

In [3]:
print(len(history), len(msgs))

108927 110675


In [43]:
class SummaryBuilder:
    MAX_TOKENS = 30000
    SUMMARY_MAX_TOKENS = 6000
    CHARS_PER_TOKEN = 2.7
    
    def __init__(self, history, msgs):
        self.history = history
        self.msgs = msgs
        self.tops, self.msgs_quotes = self.build_tops(history, msgs)

    @staticmethod
    def build_tops(history, msgs):
        msg_tops = {}     # top levels cache by reply_to ids
        tops = {}         # top levels data - counts, total_size, list of messages
        msgs_quotes = {}  # quote counts for messages
        empty_tops = set()
        
        for i, d in enumerate(reversed(history)):
            if d.empty:
                continue
            if d.reply_to_message_id is None:
                #print("new top", d)
                continue
            # get top from cache
            top = msg_tops.get(d.reply_to_message_id)
            if not top:
                # find real top
                top = d
                while top.reply_to_message_id:
                    top = msgs.get(top.reply_to_message_id)
                    if not top:
                        print("WARNING: unknown top", d.reply_to_message_id)
                        break
                if not top:
                    print("WARNING: cant find top for", d)
                    continue
                # fill cache
                msg_tops[d.reply_to_message_id] = top
                if top.id not in tops:
                    tops[top.id] = [0, 0, []]
                    if not top.empty:
                        print(f"New top: {top.id}")
                    else:
                        empty_tops.add(top.id)
            
            if d.reply_to_message_id != top.id:
                # massege is a reply
                msgs_quotes.setdefault(d.reply_to_message_id, 0)
                msgs_quotes[d.reply_to_message_id] += 1
        
            if d.text:
                tops[top.id][0] += 1
                tops[top.id][1] += len(d.text)
                tops[top.id][2].append(d)
        
        for et in empty_tops:
            del tops[et]
            
        return tops, msgs_quotes
        

    def format_data(self, thread_id, start=None, end=None):
        data = []
        tops = self.tops
        msgs = self.msgs
        msgs_quotes = self.msgs_quotes

        def get_user(m):
            return m.from_user.username or f"{m.from_user.first_name}({m.from_user.id})"
        
        for i, m in enumerate(tops[thread_id][2][start:end]):
            if m.from_user and m.text:
                msg = [f"== {m.date} {m.id} ==\n"]
                bases = []
                base = m
                depth = 0
                while base.reply_to_message_id != thread_id:
                    base = msgs.get(base.reply_to_message_id)
                    if base and base.text and base.from_user:
                        base_user = get_user(base)
                        for l in reversed(base.text.splitlines()):
                            bases.append(f"{base_user} > {l}")
                    depth += 1
                    if depth > 1:
                        break
                if bases:
                    bases = "\n".join(reversed(bases)) + "\n"
                    msg.append(bases)
                user = get_user(m)
                msg.append(f"{user}> {m.text}\n")
                if m.reactions and m.reactions.reactions:
                    for r in m.reactions.reactions:
                        msg.append(f"{r.emoji} {r.count} ")
                    msg.append("\n")
                if m.id in msgs_quotes:
                    msg.append(f"replies: {msgs_quotes[m.id]}\n")            
                msg.append("\n")
                data.append("".join(msg))
            if i == 0:
                print("Start date:", m.date)
        print("End date:", m.date)
        return data

    def gen_splits(self, data):
        max_split_len = (self.MAX_TOKENS - self.SUMMARY_MAX_TOKENS * 2)
        
        full_len = len("".join(data)) / self.CHARS_PER_TOKEN
        print("Full len:", full_len)
        if full_len < max_split_len:
            return [data]

        chunks = []
        chunk = []
        chunk_len = 0
        for m in reversed(data):
            chunk.append(m)
            chunk_len += len(m)
            if chunk_len / self.CHARS_PER_TOKEN > max_split_len:
                print("New chunk:", chunk_len / self.CHARS_PER_TOKEN)
                print("First msg:")
                print(chunk[-1])
                print("Last msg:")
                print(chunk[0])
                chunks.append(list(reversed(chunk)))
                chunk = []
                chunk_len = 0
                
        if chunk_len:
            chunks.append(list(reversed(chunk)))

        return list(reversed(chunks))
        

sb = SummaryBuilder(history, msgs)


New top: 92406
New top: 7626
New top: 7265
New top: 3389
New top: 26730
New top: 4621
New top: 25827
New top: 4705
New top: 27200
New top: 4638
New top: 2488
New top: 2340
New top: 5489
New top: 105418
New top: 4639
New top: 19091
New top: 1149
New top: 2789
New top: 2623
New top: 2776
New top: 7451
New top: 1136
New top: 2374
New top: 225506
New top: 233783
New top: 237230
New top: 242883
New top: 6865
New top: 4847
New top: 279399
New top: 319436


In [14]:
print(sb.msgs[215078])

{
    "_": "Message",
    "id": 215078,
    "empty": true
}


In [44]:
t = [(k, v[0], v[1]) for k, v in sorted(sb.tops.items(), key=lambda x: -x[1][0])]
t

[(92406, 53471, 3454523),
 (4621, 7977, 547483),
 (3389, 6074, 543211),
 (5489, 5881, 486407),
 (105418, 5138, 445590),
 (26730, 4215, 328507),
 (7626, 4013, 298081),
 (2340, 3207, 253811),
 (2776, 2136, 222463),
 (2488, 1119, 104855),
 (4705, 757, 53349),
 (2623, 709, 63539),
 (25827, 655, 66869),
 (4639, 632, 41544),
 (1149, 400, 27649),
 (19091, 307, 34999),
 (7265, 272, 27283),
 (4638, 270, 27572),
 (27200, 188, 11124),
 (319436, 145, 14163),
 (2789, 85, 17487),
 (2374, 38, 12158),
 (1136, 27, 20595),
 (7451, 17, 2188),
 (6865, 8, 2989),
 (225506, 7, 168),
 (233783, 7, 361),
 (279399, 5, 79),
 (237230, 4, 129),
 (4847, 2, 126),
 (242883, 1, 16)]

In [45]:
'''
[(92406, [32431, 2169905]), # Srachivnya
 (4621, [5423, 361251]),    # Itshechna
 (3389, [4291, 371518]),    # Pravo
 (5489, [3920, 307077]),    # Avto
 (105418, [3785, 329030]),  # Relocation
 (26730, [2938, 231552]),   # Consoli
 (7626, [2694, 194551]),    # Finances
 (2340, [2215, 178146]),    # Common Chat
 (2776, [1594, 162379]),    # Housing
 (4705, [655, 43829]),      # Culture
 (2488, [610, 61006]),      # Travel
 (25827, [569, 55870]),     # Job Search
 (2623, [533, 47695]),      # Medical
 (4639, [509, 33378]),      # Food
 (1149, [280, 21072]),      # Mental Health
 (7265, [181, 14294]),      # SecondHand
 (19091, [160, 21207]),     # Kids
 (4638, [150, 14610]),      # Pets
 (319436, [145, 14163]),    # Investments
 (27200, [143, 8621]),      # Memes
'''
thread_id = 92406

data = sb.format_data(thread_id, -1000, None)
splits = sb.gen_splits(data)
print(len(splits))


Start date: 2024-09-19 15:36:21
End date: 2024-09-22 19:28:02
Full len: 73484.44444444444
New chunk: 18019.629629629628
First msg:
== 2024-09-20 20:29:25 325035 ==
Artem(667857019) > https://www.instagram.com/indian0ch?igsh=MTBlcTI3aXh5dmNyZA==
alexzhv13 > ото зрада((( знов на білорусов піздіти не вийшло(((
Artem(667857019)> На твіторі вже поляки понавитягували роліків як воно себе поводить як мрозь
replies: 1


Last msg:
== 2024-09-22 19:28:02 326389 ==
pescaao > я ж не кажу що це неможливо, просто складно, чувак реально молодець
Konark96 > Думаю багато людей в Україні з тобою не погодяться
pescaao> П + п)


New chunk: 18053.333333333332
First msg:
== 2024-09-20 11:45:09 324275 ==
sofik0909 > Чому?
mykytasahan7 > якась частина поляків не довольна тим що українців тут)
sofik0909> Мені здається то менша частина)
replies: 1


Last msg:
== 2024-09-20 20:28:37 325034 ==
hvala_bok > Хто там шарить в акцентах рускага - то наші?
Artem(667857019) > https://www.instagram.com/indian0ch?igsh=MTBl

In [40]:
print(data[3])

== 2024-09-19 15:36:36 323270 ==
yuuzoo > Тут є люди які на ремоуті без дітей і живуть в Польщі :) замість того щоб десь тусить в екзотичних країнах
dmytro_kyiv_24 > ну у мене знайомий так у Дубаях потусив. Ось паспорт закінчився і привіт депортація на неньку
jack_bauer_d> і що думає робити?




In [41]:
l = len("".join(data))
print(l, l/2.72)

198408 72944.11764705883


In [ ]:
sys_base_prompt = open("prompt_ua.txt").read()
example_prompt = open("example_output.txt").read()
sys_prompt = f"""{sys_base_prompt}
======
{example_prompt}
======
"""

summary_prompt = """
This is a summary from previous periods:
===========
{}
===========
"""

text_prompt = """This is a history of new messages in the telegram chat:
===========
{}
===========
"""

summary = ""
for i, split in enumerate(splits):
    messages = [{"role": "system", "content": sys_prompt}]

    if i > 0:
        messages.append({"role": "user", "content": summary_prompt.format(summary)})
    messages.append({"role": "user", "content": text_prompt.format("".join(split))})
    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06",  # or use the model you prefer
        #model="gpt-4o-mini",  # or use the model you prefer
        messages=messages,
        temperature=0.3,
        max_completion_tokens=sb.SUMMARY_MAX_TOKENS,
    )
    summary = response.choices[0].message.content
    print(summary)
    print(response.usage)

print(summary)

**Топ обговорення за 19 Вересня 2024**

1. Трейдинг і інвестиції
2. Життя на ремоуті в Польщі
3. Депортація з Дубаїв
4. Криптовалютні втрати

---

**1. Трейдинг і інвестиції**

Основні учасники: Ooooooles, MikeBMW, dmytro_svko, VladyslavZotov

Стислий опис: Розмова про трейдинг та інвестиції, де учасники обговорюють реалістичність отримання високих прибутків від інвестицій. Згадується, що трейдери часто зазнають невдачі, і що очікування прибутків понад 6-10% на рік є нереалістичними.

Цитати:
- **"Кароче торгувати на ізі 30% в рік це хуйня, атвічаю"** (323271 Ooooooles)
- **"Десь читав що трейдери вилітають в трубу в дуже переважній більшості"** (323273 Ooooooles)
- **"Модна звісно думати шо ти попаде в 1%"** (323276 Ooooooles)
- **"Але розраховувати більше чим на 6-10% в рік від інвестицій це казочки"** (323279 Ooooooles)
- **"Залежить від стилю трейдера :) хтось трейдить слотами на годину-дві"** (323282 dmytro_svko)

Висновок: Учасники скептично ставляться до можливості стабільного о

In [34]:
print(response.usage)

CompletionUsage(completion_tokens=2177, prompt_tokens=8258, total_tokens=10435, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0))
